## ESA CCI Open Data Portal access using xcube

The aim of this notebook is to show how to query for all the data sets provided by the CCI ODP Store and how to get the neccessary information to create a data set cube.

To run this Notebook, make sure the ESA CCI ODP / xcube Integration is setup correctly, see [Ex0-DCFS-Setup](./Ex0-DCFS-Setup.ipynb).
We begin with installing some necessary imports.

In [1]:
from xcube_cci.dataaccess import CciOdpDataStore

In [2]:
from xcube.util.ipython import enable_asyncio
enable_asyncio()

---
Now we can create the store.

In [3]:
cci_store = CciOdpDataStore()

We can check what types of data it provides ...

In [4]:
cci_store.get_type_specifiers()

(dataset, dataset[cube])

There are many store methods which offer the parameter *type_specifier*. This parameter is optional, but if it is used, it must be one of the above. If it is not used, the default is used, which is the first entry in the tuple (i.e, 'dataset').

Now, let's have a look what data sets are available as cubes. 

In [5]:
datasets = cci_store.get_data_ids(type_specifier='dataset[cube]')
list(datasets)

['esacci.AEROSOL.climatology.L3.AAI.multi-sensor.multi-platform.MSAAI.1-7.r1',
 'esacci.AEROSOL.day.L3.AAI.multi-sensor.multi-platform.MSAAI.1-7.r1',
 'esacci.AEROSOL.day.L3C.AER_PRODUCTS.AATSR.Envisat.ADV.2-31.r1',
 'esacci.AEROSOL.day.L3C.AER_PRODUCTS.AATSR.Envisat.SU.4-3.r1',
 'esacci.AEROSOL.day.L3C.AER_PRODUCTS.ATSR-2.ERS-2.ADV.2-31.r1',
 'esacci.AEROSOL.day.L3C.AER_PRODUCTS.ATSR-2.ERS-2.SU.4-3.r1',
 'esacci.AEROSOL.day.L3C.AER_PRODUCTS.multi-sensor.multi-platform.AATSR-ENVISAT-ENS_DAILY.v2-6.r1',
 'esacci.AEROSOL.day.L3C.AOD.MERIS.Envisat.MERIS_ENVISAT.2-2.r1',
 'esacci.AEROSOL.mon.L3.AAI.multi-sensor.multi-platform.MSAAI.1-7.r1',
 'esacci.AEROSOL.mon.L3C.AER_PRODUCTS.AATSR.Envisat.ADV.2-31.r1',
 'esacci.AEROSOL.mon.L3C.AER_PRODUCTS.AATSR.Envisat.SU.4-3.r1',
 'esacci.AEROSOL.mon.L3C.AER_PRODUCTS.ATSR-2.ERS-2.ADV.2-31.r1',
 'esacci.AEROSOL.mon.L3C.AER_PRODUCTS.multi-sensor.multi-platform.AATSR-ENVISAT-ENS_MONTHLY.v2-6.r1',
 'esacci.AEROSOL.mon.L3C.AER_PRODUCTS.multi-sensor.multi-p

This might have been a bit much. In case you are looking for particular data sets, you can search for them. You can list which search options are available:

In [6]:
cci_store.get_search_params_schema()

The parameters are listed under *properties*. Let's use *ecv* and *frequency*. The parameter *type_specifier* is always supported.

In [7]:
[descriptor.data_id for descriptor in cci_store.search_data(ecv='AEROSOL', frequency='month', type_specifier='dataset[cube]')]

['esacci.AEROSOL.mon.L3.AAI.multi-sensor.multi-platform.MSAAI.1-7.r1',
 'esacci.AEROSOL.mon.L3C.AER_PRODUCTS.AATSR.Envisat.ADV.2-31.r1',
 'esacci.AEROSOL.mon.L3C.AER_PRODUCTS.AATSR.Envisat.SU.4-3.r1',
 'esacci.AEROSOL.mon.L3C.AER_PRODUCTS.ATSR-2.ERS-2.ADV.2-31.r1',
 'esacci.AEROSOL.mon.L3C.AER_PRODUCTS.multi-sensor.multi-platform.AATSR-ENVISAT-ENS_MONTHLY.v2-6.r1',
 'esacci.AEROSOL.mon.L3C.AER_PRODUCTS.multi-sensor.multi-platform.ATSR2-ENVISAT-ENS_MONTHLY.v2-6.r1',
 'esacci.AEROSOL.mon.L3C.AOD.MERIS.Envisat.MERIS_ENVISAT.2-2.r1']

'esacci.AEROSOL.mon.L3.AAI.multi-sensor.multi-platform.MSAAI.1-7.r1' sounds interesting. Let's get more information about that one. Note that the description might be slightly different depending on the parameter *type_specifier*. E.g., in case you ask for a *dataset[cube]*, you can rely on that the data variables will have (at least) dimensions *time*, *lat*, and *lon*.

In [8]:
cci_store.describe_data('esacci.AEROSOL.mon.L3.AAI.multi-sensor.multi-platform.MSAAI.1-7.r1', 'dataset[cube]')

Finally, we can have a look at which openers are provided by this store.

In [9]:
cci_store.get_data_opener_ids()

('dataset:zarr:cciodp', 'dataset[cube]:zarr:cciodp')

One of these can be used for datasets, the other one for cubes.

Depending on the opener, there might be different opening parameters. Here is how you can view these parameters for a particular dataset (both parameters are optional).

In [10]:
cci_store.get_open_data_params_schema(data_id='esacci.AEROSOL.mon.L3.AAI.multi-sensor.multi-platform.MSAAI.1-7.r1', opener_id='dataset[cube]:zarr:cciodp')

Great! Now we are all set to open some cubes. Continue in [Ex1b-DCFS-Setup](./Ex1b-DCFS-CCI_access.ipynb) ...